In [2]:
import pandas as pd
import numpy as np

In [3]:
#read in empowerment zone data
ez = pd.read_csv('Regulation_Data/EZ_EC_RC/EZs.csv')

In [4]:
ez.head()

,﻿Round_I (1993-1994),Round_II (1997-1999),Round_III (2000)
0,"Atlanta, GA","Boston, MA","Fresno, CA"
1,"Baltimore, MD","Cincinnati, OH","Jacksonville, FL"
2,"Chicago, IL","Columbia, SC","Oklahoma City, OK"
3,"Detroit, MI","Sumter, SC","Pulaski County, AR"
4,"New York, NY","Columbus, OH","San Antonio, TX"


In [5]:
ez.columns

Index(['﻿Round_I (1993-1994)', 'Round_II (1997-1999)', 'Round_III (2000)'], dtype='object')

In [6]:
#create a signle array of cities that have been empowerment zones
ez_new_df = ez['﻿Round_I (1993-1994)'].append(ez['Round_II (1997-1999)'].append(ez['Round_III (2000)']))

In [7]:
#Turn array into dataframe
ez_fixed = pd.DataFrame(ez_new_df)
ez_fixed = ez_fixed.reset_index()

In [8]:
#reset index
ez_fixed.columns = ['new_index','cities']

In [10]:
#remove duplicate cities
ez_fixed = ez_fixed.drop_duplicates()

In [11]:
#separate city and state to create the unique key in the next cell that is found in the other datasets
city_state = ez_fixed['cities'].str.split(',',expand = True)
ez_fixed['city']  =  city_state[0]
ez_fixed['state']  =  city_state[1]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [12]:
ez_fixed['city_state'] = ez_fixed['city'] + '_' + ez_fixed['state']
ez_fixed.head()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,new_index,cities,city,state,city_state
0,0,"Atlanta, GA",Atlanta,GA,Atlanta_ GA
1,1,"Baltimore, MD",Baltimore,MD,Baltimore_ MD
2,2,"Chicago, IL",Chicago,IL,Chicago_ IL
3,3,"Detroit, MI",Detroit,MI,Detroit_ MI
4,4,"New York, NY",New York,NY,New York_ NY


In [13]:
#Remove spaces and characters that could potentially be found in the city_state column
ez_fixed['city_state'] = ez_fixed['city_state'].str.replace(' ','')
ez_fixed['city_state'] = ez_fixed['city_state'].str.replace(',','')
ez_fixed['city_state'] = ez_fixed['city_state'].str.replace('.','')
ez_fixed['city_state'] = ez_fixed['city_state'].str.lower()

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to 

In [14]:
#Turn column into a list
empowerment_cities = list(ez_fixed['city_state'])

# Get Datasets

In [16]:
#years to loop through
years = list(range(2001,2015))

In [18]:
final_list = []
cities = pd.read_csv('Patents_Data/cities_with_scores.csv')
for i in years:
    
    #patent data
    patents = cities.loc[cities['Year']==int(float(i))]
    patents['city_state'] = patents['city_state'].str.replace(' ','')
    patents['city_state'] = patents['city_state'].str.replace(',','')
    patents['city_state'] = patents['city_state'].str.replace('.','')
    patents['city_state'] = patents['city_state'].str.lower()
    patents['year'] = i
    
    #sbir data
    sbir = pd.read_csv('Regulation_Data/SBIR/clean_SBIR/clean_sbir_'+str(i)+'.csv')
    sbir['City_State'] = sbir['City_State'].str.replace(' ','')
    sbir['City_State'] = sbir['City_State'].str.replace(',','')
    sbir['City_State'] = sbir['City_State'].str.replace('.','')
    sbir['City_State'] = sbir['City_State'].str.lower()
    
    #first join
    sbir_cities = patents.merge(sbir, left_on = 'city_state', right_on = 'City_State', how = 'left')
    
    #econ data
    if int(float(i)) < 2002:
        print("in 1997" + str(i))
        econ = pd.read_csv('econ_data/1997_creative_class.csv')
    if (int(float(i)) < 2007) & (int(float(i)) >= 2002):
        print("in 2002" + str(i))
        econ = pd.read_csv('econ_data/2002_creative_class.csv')
    if (int(float(i)) < 2012) & (int(float(i)) >= 2007):
        print("in 2007" + str(i))
        econ = pd.read_csv('econ_data/2007_creative_class.csv')
    if int(float(i)) >= 2012:
        print("in 2012" + str(i))
        econ = pd.read_csv('econ_data/2012_creative_class.csv')
    econ['city_state'] = econ['city_state'].str.replace(' ','')
    econ['city_state'] = econ['city_state'].str.replace('.','')
    econ['city_state'] = econ['city_state'].str.replace(',','')
    econ['city_state'] = econ['city_state'].str.lower()
    
    #second join
    sbir_creative_cities = sbir_cities.merge(econ,left_on = 'city_state', right_on = 'city_state', how = 'left')
    
    #fill sbir NaN with zero
    sbir_creative_cities['Award Mean'] = sbir_creative_cities['Award Mean'].fillna(0)
    sbir_creative_cities['Award Sum'] = sbir_creative_cities['Award Sum'].fillna(0)
    sbir_creative_cities['Company Count'] = sbir_creative_cities['Company Count'].fillna(0)
    
    #regulation data
    if i < 2001:
        sbir_creative_ff_cities = sbir_creative_cities
        
    if (i >= 2001) & (i<2013):
        performance = pd.read_csv('Regulation_Data/Federal_Funding/FF'+str(i)+'_performance.csv')
        recipient = pd.read_csv('Regulation_Data/Federal_Funding/FF'+str(i)+'_recipient.csv')
    
        performance['city_state_abrv'] = performance['primary_place_of_performance_city_name'] + '_' + performance['primary_place_of_performance_state_code']
        recipient['city_state_abrv'] = recipient['recipient_city_name'] + '_' + recipient['recipient_state_code']
    
        performance['city_state_abrv'] = performance['city_state_abrv'].str.replace(' ','')
        performance['city_state_abrv'] = performance['city_state_abrv'].str.replace(',','')
        performance['city_state_abrv'] = performance['city_state_abrv'].str.replace('.','')
        performance['city_state_abrv'] = performance['city_state_abrv'].str.lower()
    
        recipient['city_state_abrv'] = recipient['city_state_abrv'].str.replace(' ','')
        recipient['city_state_abrv'] = recipient['city_state_abrv'].str.replace(',','')
        recipient['city_state_abrv'] = recipient['city_state_abrv'].str.replace('.','')
        recipient['city_state_abrv'] = recipient['city_state_abrv'].str.lower()
    
        #third and fourth joins
        sbir_creative_performance_cities = sbir_creative_cities.merge(performance,left_on = 'City_State', right_on = 'city_state_abrv', how = 'left')
        sbir_creative_ff_cities = sbir_creative_performance_cities.merge(recipient,left_on = 'City_State', right_on = 'city_state_abrv', how = 'left')
        sbir_creative_ff_cities['performance_count'] = sbir_creative_ff_cities['performance_count'].fillna(0)
        sbir_creative_ff_cities['performance_amount'] = sbir_creative_ff_cities['performance_amount'].fillna(0)
        sbir_creative_ff_cities['recipient_count'] = sbir_creative_ff_cities['recipient_count'].fillna(0)
        sbir_creative_ff_cities['recipient_amount'] = sbir_creative_ff_cities['recipient_amount'].fillna(0)
        sbir_creative_ff_cities =  sbir_creative_ff_cities.drop(['Unnamed: 0_y.1', 'performance_city_state',
       'primary_place_of_performance_city_name',
       'primary_place_of_performance_state_code', 'city_state_abrv_x',
       'Unnamed: 0.1', 'recipient_city_state', 'recipient_city_name', 'recipient_state_code',
       'city_state_abrv_y'], axis = 1)
        
    if i >= 2013:
        sbir_creative_ff_cities = sbir_creative_cities
        
    print(i)
    
    #case when for empowerment zone
    sbir_creative_ff_cities['empowerment_zone'] =  pd.np.where(
     sbir_creative_ff_cities['City_State'].isin(empowerment_cities), 
    '1','0')
    
    #demographic data
    if int(float(i)) < 2000:
        demog = pd.read_csv('demog/1990_demographics_finished.csv')
    if (int(float(i)) < 2008) & (int(float(i)) >= 2000):
        demog = pd.read_csv('demog/2000_demographics_finished.csv')
    if int(float(i)) >= 2008:
        demog = pd.read_csv('demog/2008_demographics_finished.csv')
    
    #fifth join
    
    sbir_creative_ff_demog_cities = sbir_creative_ff_cities.merge(demog,left_on = 'city_state', right_on = 'abrv_city_state', how = 'left')
    
    #Final Cleaning
    sbir_creative_ff_demog_cities = sbir_creative_ff_demog_cities.drop(['Unnamed: 0_x', 'City_x', 'State_x', 'year', 'Unnamed: 0_y', 'City_y','City_State',
       'State_y', 'Unnamed: 0','abrv_city_state'], axis=1)
    
    #Write out the annual data aggregation into a csv
    sbir_creative_ff_demog_cities.to_csv('all_collected_data_'+str(i)+'.csv')
    
    #Append to the list to create a list of dataframes
    final_list.append(sbir_creative_ff_demog_cities)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

in 19971997
1997
in 19971998
1998
in 19971999
1999
in 19972000
2000
in 19972001
2001
in 20022002
2002
in 20022003
2003
in 20022004
2004
in 20022005
2005
in 20022006
2006
in 20072007
2007
in 20072008
2008
in 20072009
2009
in 20072010
2010
in 20072011
2011
in 20122012
2012
in 20122013
2013
in 20122014
2014
